In [22]:
# read in preplacements.csv into a dataframe

import pandas as pd
import numpy as np

df = pd.read_csv('preplacements.csv')

# print the first 5 rows of the dataframe
print(df.head())

# Dorm Mapping
# 1 = East
# 2 = North
# 3 = South
# 4 = West
# 5 = Atwood
# 6 = Sontag
# 7 = Case
# 8 = Drinkward
# 9 = Linde


           ID First Name   Last Name                  Email  \
0  40210584.0       Simi        Vora        svora@g.hmc.edu   
1  40205508.0     Alisha     Chulani     achulani@g.hmc.edu   
2  40210228.0     Carmen  Horenstein  chorenstein@g.hmc.edu   
3  40215659.0      Avery    Anderson   avanderson@g.hmc.edu   
4  40216167.0       Cate     Lewison     clewison@g.hmc.edu   

  Rising Class Yr (next year) Planned Grad Sess/Year   Dorm Room  \
0                          SR       Spring Term 2025   East  170   
1                          SR       Spring Term 2025  North  276   
2                          SR       Spring Term 2025  North  272   
3                          SO       Spring Term 2026  Linde  232   
4                          SO       Spring Term 2026  Linde  230   

  Preplacement Reason  
0                 NaN  
1             Proctor  
2        Proctor Pull  
3             Proctor  
4        Proctor Pull  


In [23]:
from typing import List, Dict
from numpy import NaN
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy.sql import text
from uuid import uuid4

# import env variables
import os
from dotenv import load_dotenv
from pathlib import Path


dotenv_path = os.path.join(os.getcwd(), '.env')
print(dotenv_path)

load_dotenv(dotenv_path=dotenv_path, verbose=True)

cloud_sql_pass = os.environ.get('CLOUD_SQL_PASS')
cloud_sql_ip = os.environ.get('CLOUD_SQL_IP')
cloud_sql_db_name = os.environ.get('CLOUD_SQL_DB_NAME')
cloud_sql_user = os.environ.get('CLOUD_SQL_USER')

CONNSTR = f'postgresql://{cloud_sql_user}:{cloud_sql_pass}@{cloud_sql_ip}/{cloud_sql_db_name}'

engine = create_engine(CONNSTR)

# print(CONNSTR)

with engine.connect() as connection:
    # loop through the dataframe and insert each row into the database
    for index, row in df.iterrows():
        # create a dictionary to store the values for each row
        values = {
            'first_name': row['First Name'],
            'last_name': row['Last Name'],
            'email': row['Email'],
            'reslife_role': row['Preplacement Reason']
        }
        # if values['reslife_role'] does not contain the word 'Proctor' or 'Mentor' then set it to 'None'
        print(values['reslife_role'])
        if str(values['reslife_role']) == "nan" or ('proctor' not in values['reslife_role'].lower() and 'mentor' not in values['reslife_role'].lower()):
            values['reslife_role'] = 'none'

        values['reslife_role'] = values['reslife_role'].lower().strip()
        year = 0
        in_dorm = 0
        draw_number = 0
        preplaced = True

        # create a query to insert the values into the database
        query = f"INSERT INTO users (first_name, last_name, draw_number, year, preplaced, in_dorm, email, reslife_role) VALUES ('{values['first_name']}', '{values['last_name']}', {draw_number}, {year}, {preplaced}, {in_dorm}, '{values['email']}', '{values['reslife_role']}')"
        # execute the query
        result = connection.execute(text(query))

        print(f'Inserted {values["first_name"]} {values["last_name"]} into the database')
    
    connection.commit()




/Users/tomqlam/workspaces/roomdraw/database/.env
nan
Inserted Simi Vora into the database
Proctor
Inserted Alisha Chulani into the database
Proctor Pull
Inserted Carmen Horenstein into the database
Proctor
Inserted Avery Anderson into the database
Proctor Pull
Inserted Cate Lewison into the database
Proctor
Inserted Lea Twicken into the database
Proctor Pull
Inserted August McIntosh-Lombardo into the database
Proctor
Inserted Amelia Acker into the database
Proctor Pull
Inserted Elena Miller into the database
Proctor
Inserted Jordan Carlin into the database
Proctor Pull
Inserted Jasper Cox into the database
Proctor
Inserted Emily Burger into the database
Proctor Pull
Inserted Claire Rafferty into the database
Proctor
Inserted Shreya Balaji into the database
Proctor Pull
Inserted Sarah Lammert into the database
Proctor Pull
Inserted Stuart Kerr into the database
Proctor
Inserted Marina Ring into the database
Mentor
Inserted Lev Gruber into the database
nan
Inserted Luke Vlases into the d

In [24]:
with engine.connect() as connection:
    # query the database to get all the users ids which were assigned by the database and insert them into the dataframe
    query = "SELECT id, first_name, last_name FROM users"
    result = connection.execute(text(query))
    # create a dictionary to store the ids of the users
    user_ids = {}
    # loop through the result and add the id to the dictionary
    for row in result:
        user_ids[f'{row[1]} {row[2]}'] = row[0]

    # loop through the dataframe and add the user ids to the dataframe
    for index, row in df.iterrows():
        df.at[index, 'id'] = user_ids[f'{row["First Name"]} {row["Last Name"]}']

    connection.commit()

In [25]:
import requests
jwt = "INSERT_JWT_HERE"

# link is localhost:8080/rooms/preplace/:roomuuid
# query the database to get all the rooms
with engine.connect() as connection:
    query = "SELECT room_uuid, dorm_name, room_id FROM rooms"
    result = connection.execute(text(query))
    # create a dictionary to store the room ids
    room_ids = {}
    # loop through the result and add the room ids to the dictionary
    for row in result:
        room_ids[f'{row[1]} {row[2]}'] = row[0]

    connection.commit()

# find all rows where the which share a common value for both Dorm and Room
# iterate through the groups that share a common value for both Dorm and Room and print the group
for name, group in df.groupby(['Dorm', 'Room']):
    # find the room_uuid for the group
    room_uuid = room_ids[f'{name[0]} {name[1]}']
    print(f'Room UUID: {room_uuid}')

    # create a rest call to the server to preplace the users in the group
    # rest request body contains json with:
    # type PreplacedRequest struct {
    # 	ProposedOccupants []int  `json:"proposedOccupants"`
    # 	UserJWT           string `json:"userJWT"`
    # }
    # use the id of the users in the group to create the proposed occupants list
    users_in_group = group['id'].tolist()
    # convert to integers
    users_in_group = [int(user) for user in users_in_group]

    print(f'Users in group: {users_in_group}')
    proposed_occupants = []
    for user in users_in_group:
        proposed_occupants.append(user)
    # create the json body for the rest request
    json_body = {
        'proposedOccupants': proposed_occupants,
        'userJWT': jwt
    }

    # create the headers for the rest request
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {jwt}'
    }

    # create the url for the rest request
    url = f'http://localhost:8080/rooms/preplace/{room_uuid}'

    print(url)

    response = requests.post(url, headers=headers, json=json_body)

    # print the response
    # print(response.json())
        


Room UUID: d182df24-1fb2-4907-8eaf-d287ada2bf0c
Users in group: [16]
http://localhost:8080/rooms/preplace/d182df24-1fb2-4907-8eaf-d287ada2bf0c
Room UUID: b40ab6ab-4fdc-43a9-a767-caf324f5f2e1
Users in group: [17]
http://localhost:8080/rooms/preplace/b40ab6ab-4fdc-43a9-a767-caf324f5f2e1
Room UUID: 53a82b18-852d-4411-9e8b-4bc47bb0a59d
Users in group: [18]
http://localhost:8080/rooms/preplace/53a82b18-852d-4411-9e8b-4bc47bb0a59d
Room UUID: 8b3bc6e6-619e-49b2-882a-24a3761b72f5
Users in group: [19, 31]
http://localhost:8080/rooms/preplace/8b3bc6e6-619e-49b2-882a-24a3761b72f5
Room UUID: 1df4ee2f-9dd2-41c3-8b19-604f8e3ccf34
Users in group: [32]
http://localhost:8080/rooms/preplace/1df4ee2f-9dd2-41c3-8b19-604f8e3ccf34
Room UUID: 8ce280a6-84e7-4cf8-bf8a-0be92a314f9f
Users in group: [33]
http://localhost:8080/rooms/preplace/8ce280a6-84e7-4cf8-bf8a-0be92a314f9f
Room UUID: 911e0e29-7177-4d4f-bdbd-e0163df75e93
Users in group: [34, 71]
http://localhost:8080/rooms/preplace/911e0e29-7177-4d4f-bdbd-e016